In [1]:
from __future__ import print_function
import datetime
from functools import reduce
import os

import pandas as pd
import numpy as np
%matplotlib nbagg
import matplotlib.pyplot as plt

In [2]:
# Data collected from a spark query at CERN, in pandas pickle format
# CRAB jobs only have data after Oct. 2017
ws = pd.read_pickle("data/working_set_day.pkl.gz")
# spark returns lists, we want to use sets
ws['working_set_blocks'] = ws.apply(lambda x: set(x.working_set_blocks), 'columns')
ws['working_set'] = ws.apply(lambda x: set(x.working_set), 'columns')

In [3]:
#   DBS BLOCKS table schema:
#     BLOCK_ID NOT NULL NUMBER(38)
#     BLOCK_NAME NOT NULL VARCHAR2(500)
#     DATASET_ID NOT NULL NUMBER(38)
#     OPEN_FOR_WRITING NOT NULL NUMBER(38)
#     ORIGIN_SITE_NAME NOT NULL VARCHAR2(100)
#     BLOCK_SIZE NUMBER(38)
#     FILE_COUNT NUMBER(38)
#     CREATION_DATE NUMBER(38)
#     CREATE_BY VARCHAR2(500)
#     LAST_MODIFICATION_DATE NUMBER(38)
#     LAST_MODIFIED_BY VARCHAR2(500)
if not os.path.exists('data/block_size.npy'):
    blocksize = pd.read_csv("data/dbs_blocks.csv", dtype='i8', usecols=(0,5), names=['block_id', 'block_size'])
    np.save('data/block_size.npy', blocksize.values)
    blocksize = blocksize.values
else:
    blocksize = np.load('data/block_size.npy')

# We'll be accessing randomly, make a dictionary
blocksize = {v[0]:v[1] for v in blocksize}

In [4]:
# join the data tier definitions
datatiers = pd.read_csv('data/dbs_datatiers.csv').set_index('id')
ws['data_tier'] = datatiers.loc[ws.d_data_tier_id].data_tier.values

In [5]:
date_index = np.arange(np.min(ws.day.values//86400), np.max(ws.day.values//86400)+1)
date_index_ts = np.array(list(datetime.date.fromtimestamp(day*86400) for day in date_index))

In [6]:
ws_filtered = ws[(ws.crab_job==True) & (ws.data_tier.str.contains('MINIAOD'))]

In [18]:
ws_filtered.head()

,day,d_data_tier_id,crab_job,input_campaign,working_set_blocks,working_set,sum_throughput,sum_walltime,njobs,data_tier
29,1505779200,31223,True,Run2017C,"{17821696, 17788929, 17735690, 17795096, 17850...","{13433184, 13414272, 13428515, 13428612, 13433...",590714.646119,5.587856e+06,6877,MINIAOD
36,1506556800,31224,True,PhaseIFall16MiniAOD,"{17459591, 17460747, 17459596, 17459597, 17459...","{13375987, 13384516, 13384517}",1.519105,7.820499e+04,310,MINIAODSIM
41,1508025600,31223,True,Run2016G,"{18001926, 18046986, 17258508, 17252365, 17899...","{13333474, 13333442, 13444164, 13444197, 13334...",714162.629017,3.321386e+07,21808,MINIAOD
43,1508630400,31223,True,Run2016B,"{17836038, 18030599, 17838091, 17836046, 17264...","{13333764, 13268166, 13434055, 13443773, 13335...",206232.139828,4.769794e+07,10422,MINIAOD
51,1510099200,31223,True,Run2016D,"{17250323, 17250326, 17250342, 17256490, 17250...","{13333477, 13333479, 13338280, 13445225, 13441...",302464.268748,5.938120e+07,19222,MINIAOD


In [19]:
ws_filtered[(ws_filtered.day==1505779200)]

,day,d_data_tier_id,crab_job,input_campaign,working_set_blocks,working_set,sum_throughput,sum_walltime,njobs,data_tier
29,1505779200,31223,True,Run2017C,"{17821696, 17788929, 17735690, 17795096, 17850...","{13433184, 13414272, 13428515, 13428612, 13433...",5.907146e+05,5.587856e+06,6877,MINIAOD
9046,1505779200,31223,True,Run2017D,"{17916416, 17921541, 17959943, 17952776, 17958...","{13439874, 13440051, 13439869, 13439878}",7.643197e+05,2.267267e+07,18018,MINIAOD
9604,1505779200,31223,True,Run2017B,"{17671040, 17671041, 17671042, 17671044, 17674...",{13405551},1.997375e+03,1.038737e+06,457,MINIAOD
11909,1505779200,31224,True,RunIISummer16MiniAODv2,"{17006592, 17006595, 17006596, 17006597, 17006...","{13295625, 13322265, 13297699, 13318182, 13291...",1.491914e+06,1.132013e+08,51608,MINIAODSIM
15557,1505779200,31223,True,Run2016E,"{16660480, 16642055, 16625671, 17248269, 16625...","{13393667, 13391204, 13264269, 13388558, 13264...",1.318742e+05,2.285176e+07,11103,MINIAOD
21476,1505779200,31223,True,Run2016D,"{17563648, 16758792, 16580620, 16625682, 17250...","{13263653, 13333479, 13392103, 13265321, 13264...",1.364554e+05,2.581424e+07,11752,MINIAOD
21739,1505779200,31223,True,Run2016H,"{17633280, 17633281, 17553410, 16781315, 17633...","{13400416, 13271072, 13243845, 13333510, 13333...",2.892749e+05,2.799518e+07,15671,MINIAOD
23714,1505779200,31224,True,PhaseIFall16MiniAOD,"{17460747, 17459500, 17459508, 17460935}","{13384516, 13384517}",2.596650e+00,9.173924e+02,21,MINIAODSIM
26561,1505779200,31223,True,Run2016G,"{16654340, 16721927, 16717831, 16621579, 17258...","{13264768, 13333474, 13333730, 13333348, 13391...",2.250373e+05,5.582330e+07,20905,MINIAOD
31961,1505779200,31224,True,RunIISummer17MiniAOD,{17975544},{13445360},2.673970e+00,7.052930e+04,5,MINIAODSIM


In [20]:
ws_filtered[(ws_filtered.day==1505779200)].working_set_blocks

29       {17821696, 17788929, 17735690, 17795096, 17850...
9046     {17916416, 17921541, 17959943, 17952776, 17958...
9604     {17671040, 17671041, 17671042, 17671044, 17674...
11909    {17006592, 17006595, 17006596, 17006597, 17006...
15557    {16660480, 16642055, 16625671, 17248269, 16625...
21476    {17563648, 16758792, 16580620, 16625682, 17250...
21739    {17633280, 17633281, 17553410, 16781315, 17633...
23714             {17460747, 17459500, 17459508, 17460935}
26561    {16654340, 16721927, 16717831, 16621579, 17258...
31961                                           {17975544}
32789    {17553409, 16637963, 16635917, 16627730, 17260...
39787    {17258499, 16664585, 16705556, 17250329, 17254...
50823    {16762881, 16762882, 17524739, 16758791, 17264...
53260    {17819891, 17867575, 17819894, 17819895, 17712...
53836    {17505794, 17505795, 17505796, 17506572, 17506...
53837    {15073287, 15130641, 15177753, 15177756, 15104...
Name: working_set_blocks, dtype: object

In [21]:
blocks_day = []
for i, day in enumerate(date_index):
    today = (ws_filtered.day==day*86400)
    blocks_day.append(reduce(lambda a,b: a.union(b), ws_filtered[today].working_set_blocks, set()))

print("Done assembling blocklists")


Done assembling blocklists


In [23]:
block_dict = {}
i=0
for el in blocks_day:
    i=i+1
    if len(el)>0:
        block_dict[i] = el

In [26]:
block_dict.keys()

dict_keys([237, 238, 239, 240, 241, 242, 243, 244, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444,

In [ ]:
block_dict

In [31]:
len(block_dict[481]-block_dict[480])

6812

In [32]:
len(block_dict[481].intersection(block_dict[480]))

5474

In [33]:
len(block_dict[481].intersection(block_dict[480]).intersection(block_dict[482]))

3915

In [37]:
b=block_dict[450]
i=450
print("%i, %i"%(i-450,len(b)))
for i in range(451,500):
    b=b.intersection(block_dict[i])
    print("%i, %i"%(i-450,len(b)))

0, 9010
1, 5251
2, 3514
3, 2752
4, 2123
5, 2115
6, 1973
7, 1303
8, 1277
9, 1265
10, 921
11, 806
12, 779
13, 757
14, 187
15, 184
16, 158
17, 156
18, 156
19, 156
20, 156
21, 156
22, 156
23, 156
24, 156
25, 156
26, 154
27, 67
28, 11
29, 11
30, 5
31, 5
32, 5
33, 4
34, 4
35, 3
36, 3
37, 3
38, 3
39, 3
40, 3
41, 3
42, 3
43, 3
44, 3
45, 3
46, 3
47, 3
48, 2
49, 2
